### Saving and restoring variables
TensorFlow 变量提供一个最好的方式来代表被你程序共享的，持续的状态(represent shared, persistent state manipulated by your program). 这一部分介绍怎么样保存和恢复变量。注意到 Estimators自动的保存和恢复变量(在model_dir)中间。

__tf.train.Saver__ 类提供了保存和回复models的方法(method)。 __tf.train.Saver__ 构造器(constructor) 添加了graph中所有的，或者是特定的lists的variable的 __save__ 和 __restore__ 到graph当中的ops。 __Saver__ 对象提供了可以运行这些 ops 的方法(methods)， 制定checkpoints文件写和读取的路径。

saver会保存所有的已经在model中定义的所有变量。在载入一个模型的时候并不知道怎么样来构建它的graph的（例如，写一个一般载入模型的程序），那么可以查看本问下方 Overview of saving and restoring model的部分。

TensorFlow在二进制(binary) checkpoint files保存变量，大致来说，就是把变量名字映射到tensor值。

#### Saving variables
使用 __tf.train.Saver()__ 创建一个 __Saver__ 可以管理model当中的所有变量。例如，下面的code示例怎么样调用 __tf.train.Saver.save__ 方法保存变量到checkpoint files：

In [6]:
import tensorflow as tf
# Create some variables
v1 = tf.get_variable("v1",shape=[3], initializer=tf.zeros_initializer)
v2 = tf.get_variable("v2",shape=[5], initializer=tf.zeros_initializer)

inc_v1 = v1.assign(v1+1)
dev_v2 = v2.assign(v2-1)

# Add an op to initialize the variables.
init_op = tf.global_variables_initializer()

# Add ops to save and restore all the variables
saver = tf.train.Saver()

# Later,launch the model, initialize the variables, do some work,and save 
# the variables to disk
with tf.Session() as sess:
    sess.run(init_op)
    # Do some work with the model
    inc_v1.op.run()
    dev_v2.op.run()
    # Save the variables to disk
    save_path = saver.save(sess,'/tmp/model.cpkt')
    print("Model saved in path: %s" % save_path)

Model saved in path: ./tmp/model.cpkt


#### Restoring variables
__tf.train.Saver__ 对象并不仅仅只是将变量储存到checkpoint files，它还能够恢复（restore）变量。当恢复之前没有提前初始化的变量的时候，以下的code表明怎么样使用 __tf.train.Saver.restore__ 方法来从checkpoint files 恢复变量。
> __Note：__  
* 注意到事实上没有一个本地文件是"/tmp/model.ckpt", 这是为checkpoint文件名创造的前缀(prefix)。用户只需要用本地checkpoint文件的前缀来与其交互( Users only interact with the prefix instead of physical checkpoint files)

In [ ]:
tf.reset_default_graph()

# Create some varibales
v1 = tf.get_variable("v1",shape=[3])
v2 = tf.get_variable("v2",shape=[5])

# Add ops to save and restore all the variables
saver = tf.train.Saver()

# Later, lauch the model, use the saver to restore variables form disk,
# and do some work with the model
with tf.Session() as sess:
    # Restore variables form disk.
    saver.restore(sess,'/tmp/model.cpkt')
    print("Model restored.")
    # Check the values of the variables
    print("v1: %s" % v1.eval())
    print("v2: %s" % v2.eval())

INFO:tensorflow:Restoring parameters from /tmp/model.cpkt


#### Choosing which variables to save and restore
如果没有传递任何参数到 __tf.train.Saver()__ ， saver 处理graph当中的所有的变量。每个变量在创建的时候都会被储存到传递时使用的名字。

在checkpoint文件中显示的指定变量的名字有时是非常有用的。例如，你可以训练有一个命名为"weights"的变量名，但是你想要将这个值储存到一个名字为"params"的变量中。

有的时候只存储和恢复被model使用的变量的子集也是有用的。例如，你已经训练了一个五层的神经网络，但是你现在想要重新训练一个六层的新模型并且利用之前训练好的五层的weights。你可以利用saver来恢复前五层的权重。

可以通过两种方式传参递到 __tf.train.Saver()__ 构造器(constructor)来保存和载入制定的名字和变量。(You can easily specify the names and variables to save or load by passing to the tf.train.Saver() constructor either of the following):
* a list of variables(可以存储在他们自己的名字下面）
* a Python dictionary ,keys 是使用的名字，values是管理的变量。

> __Note:__   
* 可以依照所需要save和restore模型变量的不同子集创建足够多的 __Saver__ 对象。同样的变量可以被listed到多个saver对象当中；它得值只有在 __Saver.restore()__ 方法运行得时候才会改变。
* 在session开始得时候只恢复(restore) model变量得子集，则必须要初始化其他变量得op。查看 __tf.variables_initializer__ 初始化变量。
* 查看在checkpoint里面variables，可以使用 __inspect_checkpoint__ 库，特别是 __print_tensors_in_checkpoint_file__ 函数。
* 默认状态， __Saver__ 利用每个变量 __tf.Variable.name__ 特性(property)的值。但是，当创建一个 __Saver__ 对象，可以自主选择checkpoint files里面的variables的名称。

In [ ]:
tf.reset_default_graph()
# Create some variables.
v1 = tf.get_variable("v1",[3],initializer=tf.zeros_initializer)
v2 = tf.get_variable("v2",[5],initializer=tf.zeros_initializer)

# Add ops to save and restore only "v2" using the name "v2"
saver = tf.train.Saver({"v2":v2})

# Use the saver object normally after that.
with tf.Session() as sess:
    # Initialize v1 since the saver will not.
    v1.initializer.run()
    saver.restore(sess,"/tmp/model.cpkt")
    
    print("v1: %s" % v1.eval())
    print("v2: %s" % v2.eval())

INFO:tensorflow:Restoring parameters from /tmp/model.cpkt


#### Inspect variables in a checkpoint
使用 __inspect_checkpoint__ 可以快速的探查(inspect)在checkpoint当中的variables。  
继续使用上面的例子：

In [ ]:
# import the inspect_checkpoint library
from tensorflow.python.tools import inspect_checkpoint as chkp

# print all tensors in checkpoint file
chkp.print_tensors_in_checkpoint_file("/tmp/model.cpkt",tensor_name='',
                                    all_tensors=True)
# tensor_name:  v1
# [ 1.  1.  1.]
# tensor_name:  v2
# [-1. -1. -1. -1. -1.]

# print only tensor v1 in checkpoint file
chkp.print_tensors_in_checkpoint_file("/tmp/model.cpkt",tensor_name='v1',
                                    all_tensors=False)
# tensor_name:  v1
# [ 1.  1.  1.]

# print only tensor v2 in checkpoint file
chkp.print_tensors_in_checkpoint_file("/tmp/model.cpkt",tensor_name="v2",
                                    all_tensors=False)
# tensor_name:  v2
# [-1. -1. -1. -1. -1.]

### OVerview of saving and restoring models
在想要存储和载入variables,graph,graph's metadata，更一般的，当想要save和restore模型的时候，提倡使用 SavedModel。 __SavedModel__ 是语言中立的，可恢复，封闭序列化的格式（SavedModel is a language-neutral, recoverable, hermetic serialization format）。 SavedModel保证higher-level系统和工具来produce，consume，transform TensorFlow模型。TensorFlow提供一些与SavedModel交互的机制，包括： tf.saved_model APIs, Estimator APIs 和 CLI。

### APIs to build and load a SavedModel
这一部分专注于创建和载入一个 SavedModel APIs，特别是用 low-level TensorFlow APIs。
#### Building  a SavedModel
我们提供一个SavedModel builder的Python实现。 __SavedModelBuilder__ 类提供保存多个 __MetaGraphDef__ 功能。 __MetaGraph__ 是一个dataflow graph，外加相关的变量，assets和signatures。 __MetaGraphDef__ 表示MetaGraph的协议缓冲区(protocol buffer)。 __signature__ 是graph的一系列的输入和输出。
如果说assets需要保存并且写或者拷贝到磁盘中，可以通过添加第一个 __MetaGraphDef__ 做到。如果多个 __MetaGraphDef__ are asscociated with an asset of the same name， 则只有第一个版本会被保存。

每个 __MetaGraphDef__ 必须要标注有用户特定的tags才能添加到SavedModel中。这些tags提供识别特定的 __MetaGraphDef__ 和一些列的变量和assets载入和存储的方式。这些tags一般用 __MetaGraphDef__ 的功能进行标注（例如，serving或者training),或者选择特定硬件方面(例如，GPU).

例如，以下例子中通过使用 __SavedModelBuilder__ 来创建一个 SavedModel

In [ ]:
export_dir = ...
...
bulder = tf.saved_model.builder.SavedModelBuilder(export_dir)
with tf.Session(graph=tf.Graph()) as sess:
    ...
    builder.add_meta_graph_and_variables(sess,[tag_constant.TRAINING],
                                        signature_def_map=foo_signatures,
                                        assets_collection=foo_assets)
...
# Add a second MetaGraphDef for inference
with tf.Session(graph=tf.Graph()) as sess:
    ...
    builder.add_meta_graph([tag_constants.SERVING])
...
builder.save()

#### Loading a SavedModel in Python
Python 版本的 SavedModel loader为一个 SavedModel提供载入和恢复能力。 __load__ operation 要求使用以下的信息：
* 恢复(restore) graph定义和变量的session
* 用来确定(identify)载入的MetaGraphDef的标签(tags)
* 储存 SavedModel的位置(目录)
在载入中，作为特定 MetaGraphDef部分的variables,assets和signatures会被恢复到提供的session中。

In [ ]:
export_dir = ...
...
with tf.Session(graph=tf.Graph()) as sess:
    tf.saved_model.loader.load(sess,[tag_constants.TRAINING],
                              export_dir)
    ...   

#### Loading a Savedmodel in C++
The C++ version of the SavedModel loader provides an API to load a SavedModel from a path, while allowing SessionOptions and RunOptions. You have to specify the tags associated with the graph to be loaded. The loaded version of SavedModel is referred to as SavedModelBundle and contains the MetaGraphDef and the session within which it is loaded.

const string export_dir = ...  
SavedModelBundle bundle;  
...  
LoadSavedModel(session_options, run_options, export_dir,   {kSavedModelTagTrain},
               &bundle);

#### Standard constants
SavedModel 提供大部分的 use-cases灵活的创建和载入TensroFlow graphs。在大部分常见的用户场景(use-cases)中, SavedModel's API提供了一些列的常量可以在Python和C++中容易的被不同的工具重复的使用和共享
#### Standard MetaGraphDef tags
你可以自己使用独立的tags来识别储存在SavedModel中的 __MetaGraphDef__ 。经常被使用的tags的一个自己被定义在：
* [Python](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/saved_model/tag_constants.py)
* [C++](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/cc/saved_model/tag_constants.h)

#### Standard SignatureDef constants
[SignatureDef](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/protobuf/meta_graph.proto)是定义被graph支持的计算的签名(signature)的协议缓冲区。一般用来命名input keys,output keys,method的名字定义在：
* [Python](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/saved_model/signature_constants.py)
* [C++](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/cc/saved_model/signature_constants.h)

### Using SavedModel with Estimators
训练完一个 Estimator 模型之后，可能需要创建一个服务来让这个模型接收一个请求返回一个结果。这样的服务可以在machine上本地运行也可以在云端扩展部署。
要准备这样一个已经训练好的Estimator能够serving，就需要导出一个标准的SavedModel的格式。这一部分将解释怎样：
* 制定输出节点和对应的可以服务的APIs(Classify,Regress,Predict).
* 导出模型到SavedModel format
* 从本地服务器Serve the model并且请求预测值。

#### Preparing serving inputs
在训练的过程当中， __input_fn()__ 消化(ingests)数据，准备好被模型使用。在serving的时候，同样的， __serving_input_receiver_fn()__ 接受推断请求并且准备好作为模型的数据输入，这个函数有以下的目的：
* 为graph添加placeholders用来作为serving system的推断请求的输入。
* 添加将数据从输入格式转换成model所需要的特征 Tensors的ops。

这个函数会返回一个 __tf.estimator.export.ServingInputReceiver__ 对象，将占位符(placeholders)和得到的特征Tensor打包在一起。

一般的情况是推断请求通过一个序列化的 __tf.Example__ s到达， 因此， __serving_input_receiver_fn()__ 创建单独的字符串占位符(placeholder)来接收他们。 __serving_input_receiver_fn()__ 通过添加一个 __tf.parse_example__ 操作到graph来解析 __tf.Example__ s.

当写 __serving_input_receiver_fn()__ 时，必须要传递一个解析说明(parsing specification)到 __tf.parse_example__ 告诉解释器什么样的特征名字时想要映射到Tensors上面的。 解析说明有一个字典的形式，从特征名称到 __tf.FixedFeature, tf.VarLenFeature, tf.SparseFeature__ 。注意解析说明不包含任何的label和权重columns，因为这些在serving的时候都是不提供的——不同于training的时候的用于 __input_fn()__ 的解析说明。  
结合起来，示例如下：

__tf.estimator.export.build_parsing_serving_input_receiver_fn__ utility 函数为一般的情况下提供了input receiver。
> __Note:__ 当训练一个模型用本地server的predict API服务的时候，不需要解析步骤，因为模型会直接接收raw feature data

即使不要求解析或者是其他的输入处理，也就是说，如果说serving system 可以直接输入feature Tensors，也一定要提供一个 __serving_input_receiver_fn()__ 函数来为feature Tensor创建placeholders并且传递他们。 __tf.estimator.export.build_raw_serving_input_receiver_fn__ utility为此提供解决方式。

如果这些 utilities 不满足需求，也可以自己写 __serving_input_receiver_fn()__ 函数。一种情况是在训练的时候 __input_fn()__ 包含了一些预处理逻辑在serving的时候也必须概括进来。为了减少training-serving倾斜(skew)的风险，建议分装这些预处理的过程到一个函数当中，并且在 __input_fn()__ 和 __serving_input_receiver_fn()__ 里面都调用这个函数。

注意到 __serving_input_receiver_fn()__ 也决定signature的输入比例。也就是说，当写一个 __serving_input_receiver_fn()__ ,你必须要告诉解释器，哪些signatures是需要的，并且怎么样将他们对应到你model所想要的输入中。相反的是，signature的输出比例是由model自己决定的。


In [ ]:
feature_spec = {'foo':tf.FixedFeature(...),
               'bar':tf.VarLenFeature(...)}

def serving_input_receiver_fn():
    """An input receiver that expects a serialized tf.Example."""
    serialized_tf_example = tf.placeholder(dtype=tf.string,
                                          shape=[default_batch_size],
                                          name='input_example_tensor')
    receiver_tensors = {'examples':serialized_tf_example}
    features = tf.parse_example(serialized_tf_example,feature_spec)
    return tf.estimator.export.ServingInputReceiver(features,receiver_tensors)

#### Performing the export
调用 __tf.estimator.Estimator.export_savemodel__ 导出训练好的Estimator会导出基本路径(base path)和 __serving_input_receiver_fn__ 。

示例如下：

第一次调用 __serving_input_receiver_fn()__ 创建一个新的 graph 来获得特征 Tensors, 并且通过调用这个 Estimator 的 __model_fn()__ 来基于这些特征生成model graph。这开始于一个新的Session，默认的情况下，从最近的checkpoint恢复进来(如果需要，也可以传递不同的checkpoint进来)。最后，它从给定的 __export_dir_base(i.e.,export_dir_base/<timestamps>)__ 创建一个时间戳导出目录，并且从这个Session中将包含一个单一的 MetaGraphDef的SavedModel写进这个目录。
> __Note:__ 从旧的导出当中进行垃圾收集是你的责任。否则，持续的导出会在 __export_dir_base__ 中增加 

In [ ]:
estimator.export_savedmodel(export_dir_base, serving_input_receiver_fn)

#### Specifying the outputs of a custom model
当写一个自定义的 __model_fn__ 的时候，必须要用 __tf.estimator.EstimatorSpec__ 返回值来填充 __export_outputs__ 元素。这是一个字典， __{name:output}__ 用来描述在serving的时候被导出的输出的 signatures。

通常情况下单一预测的时候，这个字典包含一个元素， __name__ 是无所谓的(immaterial)。在multi-headed模型当中，每个head代表这个dict当中的一个实体。在这种情况下， __name__ 是由你选择并且用于在serving时候可以请求特定的head的。

每一个 __output__ 的值必须由一个 __ExportOutput__ 对象例如：
__tf.estimator.export.ClassificationOutput,tf.estimator.export.RegressionOutput, or tf.estimator.export.PredictOutput。

这些输出类型直接映射到TensorFlow Serving APIs，并且决定哪些类型将遵守(honored)。
> __Note:__ 在multi-head情况中， __SignatureDef__ 会被 __export_outputs__ 字典当中的每个元素生成丙炔从model_fn返回，用同样的keys命名。这些 __SignatureDef__ s只在他们呢的输出当中有差别，就像提供相应的 __ExportOutput__ 实体。 这些输入总是被 __serving_input_receiver_fn__ 提供。 一个推断的请求可能会指定head的名字。 一个head必须要用 __[signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY](https://github.com/tensorflow/tensorflow/blob/r1.5/tensorflow/python/saved_model/signature_constants.py)__ 命名表明当一个推断没有指定的时候 __SignatureDef__ 也会被服务。

#### Serving the exported model locally
在本地配置中，可以通过 [TensorFlow Serving](https://github.com/tensorflow/serving) 来serve model, 一个开源的项目 载入一个SavedModel并且展示它作为[gRPC](https://grpc.io/)服务。
首先，[安装TensorFlow Serving](https://github.com/tensorflow/serving)

然后创建和运行本地的model server， 用你导出SavedModel的路径替换 __$export_dir_base__ 

通过下面的代码，你可以由一个server通过gRPC端口9000来进行推断请求：

In [ ]:
bazel build //tensorflow_serving/model_servers:tensorflow_model_server
bazel-bin/tensorflow_serving/model_servers/tensorflow_model_server --port=9000 --model_base_path=$export_dir_base

#### Requesting predictions from a local server
依照 [PredictionService](https://github.com/tensorflow/serving/blob/master/tensorflow_serving/apis/prediction_service.proto#L15) gRPC API service定义，server响应 gRPC的请求（嵌套的协议缓存定义在不同的 [neighboring files](https://github.com/tensorflow/serving/tree/master/tensorflow_serving/apis)

从API service定义中， gRPC框架生成一个多语言的客户库提供远程接入API。在一个项目中利用 Bazel创建工具，这些库可以通过这些依赖库(dependencies)被自动的创建和提供，python的示例如下1：

python 的用户(client) code，导入库如2：

然后再通过组装一个包含请求数据的协议缓存，并且将其传递到服务桩(service stub)完成一个发送gRPC协议请求。注意怎么样创建一个空的请求协议缓存区并且通过[一般的协议缓存API](https://developers.google.com/protocol-buffers/docs/reference/python-generated?hl=zh-cn)来填充.示例如3： 

再这个示例当中会返回一个 __ClassificationResponse__ 的协议缓存。

这只是一个骨架型示例，详情参见[TensorFlow Serving](https://github.com/tensorflow/serving) 文档，详细参见 [examples](https://github.com/tensorflow/serving/tree/master/tensorflow_serving/example)

> __Note: ClassificationRequest__ 和 __RegressionRequest__ 包含一个 __tensorflow.serving.Input__ 协议缓存，转换为包含一个 __tensorflow.Example__ 的协议缓存。 __PredictRequest__ ，相反的，包含一个将特征名称对应到值得映射并且通过 __ TensorProto__ 编码。 相应得，当使用 __Classify__ 和 __Regress__ APIs得时候， TensorFlow serving输入一个序列化得 __tf.Example__ s到graph，所以 __serving_input_receiver_fn()__ 必须包含 __tf.parse_example()__ Op。 当使用一般的 __Predict__ API的时候，尽管TensorFlow serving输入 raw 特征数据到graph，也需要通过 __serving_input_receiver_fn()__ 传递

In [ ]:
# 1：
# dependencies  
    deps = [
    "//tensorflow_serving/apis:classification_proto_py_pb2",
    "//tensorflow_serving/apis:regression_proto_py_pb2",
    "//tensorflow_serving/apis:predict_proto_py_pb2",
    "//tensorflow_serving/apis:prediction_service_proto_py_pb2"
  ]


# 2：
# Python client code can then import the libraries thus:
from tensorflow_serving.apis import classification_pb2
from tensorflow_serving.apis import regression_pb2
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2

# 3:
from grpc.beta import implementations

channel = implementations.insecure_channel(host, int(port))
stub = prediction_service_pb2.beta_create_PredictionService_stub(channel)

request = classification_pb2.ClassificationRequest()
example = request.input.example_list.examples.add()
example.features.feature['x'].float_list.value.extend(image[0].astype(float))

result = stub.Classify(request, 10.0)  # 10 secs timeout

### CLI to inspect and execute SavedModel
略，详情参见 [link](https://www.tensorflow.org/programmers_guide/saved_model?hl=zh-cn)